# Opening a Coffee Shop in Honolulu, Hawaii

<b>Author: Jamie Fraser<b>

### Preprocess Data

In [1]:
#Import required packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
import io
from pandas.io.json import json_normalize
import matplotlib as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from bs4 import BeautifulSoup
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         247 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  python_abi         conda-forge/linux-64::python_abi-3.7-2_cp37m

The following p

In [2]:
#Import Hawaii Zip Code Data from GitHub
HIZip = pd.read_csv('https://raw.githubusercontent.com/jamiemfraser/Capstone/main/HIZip.csv')
HIZip.head()

,ZIP,latitude,longitude
0,96701,21.406053,-157.884952
1,96703,22.145578,-159.385681
2,96704,19.337269,-155.837622
3,96705,21.899532,-159.568546
4,96706,21.344782,-158.022298


### Exploratory Data Analysis

In [3]:
address = 'Honolulu, Hawaii, USA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Hawaii are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Hawaii are 21.304547, -157.855676.


In [4]:
#Create a map of Hawaii and visualize the ZIP codes
map_HI = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, zips in zip(HIZip['latitude'], HIZip['longitude'], HIZip['ZIP']):
    label = '{}'.format(zips)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_HI)  
    
map_HI

In [ ]:
### 

In [5]:
#Initialize Foursquare credentials
CLIENT_ID = '40WZZPJ0DT0VE22KVJP5QQWPVUWJPNFWZNGYM4K0OZYJAYSB'
CLIENT_SECRET = 'AG41UQ403HCGXTDPQDKVFVYQRH1IGBZMI2PAM5NVGD52XCNQ'
ACCESS_TOKEN = 'K0KQG30EIJX2NULLKKDFKUI3UI5VX0IJ5WFJLJFEU0XAMEB1'
VERSION = '20180605'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 40WZZPJ0DT0VE22KVJP5QQWPVUWJPNFWZNGYM4K0OZYJAYSB
CLIENT_SECRET:AG41UQ403HCGXTDPQDKVFVYQRH1IGBZMI2PAM5NVGD52XCNQ


In [6]:
#Obtain data on Foursquare venues in Honolulu
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ZIP', 
                  'ZIP Latitude', 
                  'ZIP Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
#Return all venues in each ZIP code in Hawaii
HI_venues=getNearbyVenues(names=HIZip['ZIP'], latitudes=HIZip['latitude'], longitudes=HIZip['longitude'])

96701
96703
96704
96705
96706
96707
96708
96710
96712
96713
96714
96716
96717
96719
96720
96722
96725
96726
96727
96728
96729
96730
96731
96732
96734
96737
96738
96740
96741
96742
96743
96744
96746
96747
96748
96749
96750
96751
96752
96753
96754
96755
96756
96757
96759
96760
96761
96762
96763
96764
96765
96766
96768
96769
96770
96771
96772
96773
96774
96776
96777
96778
96779
96780
96781
96782
96783
96785
96786
96789
96790
96791
96792
96793
96795
96796
96797
96799
96813
96814
96815
96816
96817
96818
96819
96821
96822
96825
96826
96850
96853
96857
96859
96860
96863


In [8]:
#Examine the resulting dataframe
print(HI_venues.shape)
HI_venues.head()

(347, 7)


,ZIP,ZIP Latitude,ZIP Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,96701,21.406053,-157.884952,H3 Look Out,21.405143,-157.882276,Trail
1,96706,21.344782,-158.022298,Thomas H. Gentry Community pool,21.345604,-158.022541,Pool
2,96706,21.344782,-158.022298,Ewa by Gentry Community Center,21.345525,-158.022730,Recreation Center
3,96706,21.344782,-158.022298,Bus Stop 1097,21.342294,-158.025094,Bus Station
4,96706,21.344782,-158.022298,area808garage,21.343548,-158.026413,Motorcycle Shop


In [9]:
#Examine how many venues are in each ZIP code
HI_venues.groupby('ZIP').count().head()

,ZIP Latitude,ZIP Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
ZIP,,,,,,
96701,1,1,1,1,1,1
96706,5,5,5,5,5,5
96707,4,4,4,4,4,4
96732,10,10,10,10,10,10
96734,2,2,2,2,2,2


In [10]:
#Obtain the most common venue types for each ZIP code
HI_onehot = pd.get_dummies(HI_venues[['Venue Category']], prefix="", prefix_sep="")
HI_onehot['ZIP'] = HI_venues['ZIP'] 
fixed_columns = [HI_onehot.columns[-1]] + list(HI_onehot.columns[:-1])
HI_onehot = HI_onehot[fixed_columns]
HI_onehot['Café/CoffeeShop'] = HI_onehot['Café'] + HI_onehot['Coffee Shop']
HI_onehot.sort_values('Café/CoffeeShop', ascending=False).head()

,ZIP,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Baseball Field,Basketball Court,Beach,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Dog Run,Donut Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Indian Restaurant,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lingerie Store,Liquor Store,Locksmith,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Post Office,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Surf Spot,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volcano,Wine Bar,Women's Store,Yoga Studio,Zoo,Café/CoffeeShop
46,96813,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
87,96814,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
161,96815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
162,96815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
288,96850,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [25]:
#Group by ZIP code, and take the mean frequency occurance of each category
HI_grouped = HI_onehot.groupby('ZIP').mean().reset_index()
HI_grouped.head()

,ZIP,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Baseball Field,Basketball Court,Beach,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Dog Run,Donut Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Indian Restaurant,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lingerie Store,Liquor Store,Locksmith,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Post Office,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Surf Spot,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volcano,Wine Bar,Women's Store,Yoga Studio,Zoo,Café/CoffeeShop
0,96701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,96706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,96707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0

In [12]:
#Obtaine the top 5 most common venue types for each ZIP code
num_top_venues = 5

for hood in HI_grouped['ZIP']:
    temp = HI_grouped[HI_grouped['ZIP'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['ZIP']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

HI_venues_sorted = pd.DataFrame(columns=columns)
HI_venues_sorted['ZIP'] = HI_grouped['ZIP']

for ind in np.arange(HI_grouped.shape[0]):
    HI_venues_sorted.iloc[ind, 1:] = return_most_common_venues(HI_grouped.iloc[ind, :], num_top_venues)

HI_venues_sorted.head()

,ZIP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,96701,Trail,Café/CoffeeShop,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Field,Fast Food Restaurant
1,96706,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
2,96707,Italian Restaurant,Campground,Chinese Restaurant,Mountain,Café/CoffeeShop,Food,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop
3,96732,Fast Food Restaurant,German Restaurant,Diner,Pizza Place,Japanese Restaurant,Restaurant,Mexican Restaurant,Candy Store,Thrift / Vintage Store,Café/CoffeeShop
4,96734,Deli / Bodega,Korean Restaurant,Fast Food Restaurant,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Field


### Cluster ZIP Codes

In [13]:
#Use K-means to cluster the zip codes into 10 clusters, based on the predominent venue types
kclusters = 10
HI_grouped_clustering = HI_grouped.drop('ZIP', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(HI_grouped_clustering)
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 7, 6, 0, 0, 4, 9], dtype=int32)

In [14]:
#Generate new dataframe with cluster labels
HI_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
HI_merged = HI_venues
HI_merged = HI_merged.join(HI_venues_sorted.set_index('ZIP'), on='ZIP')
HI_merged = HI_merged.dropna()
HI_merged['Cluster Labels'] = HI_merged['Cluster Labels'].astype(int)
HI_merged.head()

,ZIP,ZIP Latitude,ZIP Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,96701,21.406053,-157.884952,H3 Look Out,21.405143,-157.882276,Trail,3,Trail,Café/CoffeeShop,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Field,Fast Food Restaurant
1,96706,21.344782,-158.022298,Thomas H. Gentry Community pool,21.345604,-158.022541,Pool,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
2,96706,21.344782,-158.022298,Ewa by Gentry Community Center,21.345525,-158.022730,Recreation Center,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
3,96706,21.344782,-158.022298,Bus Stop 1097,21.342294,-158.025094,Bus Station,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
4,96706,21.344782,-158.022298,area808garage,21.343548,-158.026413,Motorcycle Shop,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court


In [15]:
#Visualize the clusters in Honolulu
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(HI_merged['Venue Latitude'], HI_merged['Venue Longitude'], HI_merged['ZIP'], HI_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [16]:
#Describe cluster 0, the largest cluster
HI_merged.loc[HI_merged['Cluster Labels'] == 0]

,ZIP,ZIP Latitude,ZIP Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,96706,21.344782,-158.022298,Thomas H. Gentry Community pool,21.345604,-158.022541,Pool,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
2,96706,21.344782,-158.022298,Ewa by Gentry Community Center,21.345525,-158.022730,Recreation Center,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
3,96706,21.344782,-158.022298,Bus Stop 1097,21.342294,-158.025094,Bus Station,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
4,96706,21.344782,-158.022298,area808garage,21.343548,-158.026413,Motorcycle Shop,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
5,96706,21.344782,-158.022298,Fort Weaver Rd & Kolowaka Dr.,21.341317,-158.024955,Intersection,0,Intersection,Bus Station,Motorcycle Shop,Pool,Recreation Center,Asian Restaurant,Field,Fried Chicken Joint,Food Truck,Food Court
6,96707,21.363106,-158.082206,Upper Makakilo,21.362993,-158.082687,Mountain,0,Italian Restaurant,Campground,Chinese Restaurant,Mountain,Café/CoffeeShop,Food,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop
7,96707,21.363106,-158.082206,Panda Express,21.362005,-158.083618,Chinese Restaurant,0,Italian Restaurant,Campground,Chinese Restaurant,Mountain,Café/CoffeeShop,Food,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop
8,96707,21.363106,-158.082206,Assaggio Ristorante Italiano,21.361997,-158.083596,Italian Restaurant,0,Italian Restaurant,Campground,Chinese Restaurant,Mountain,Café/CoffeeShop,Food,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop
9,96707,21.363106,-158.082206,Camp Timberline,21.365444,-158.085569,Campground,0,Italian Restaurant,Campground,Chinese Restaurant,Mountain,Café/CoffeeShop,Food,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop
10,96732,20.883783,-156.462641,Maui Specialty Chocolates,20.885631,-156.461221,Candy Store,0,Fast Food Restaurant,German Restaurant,Diner,Pizza Place,Japanese Restaurant,Restaurant,Mexican Restaurant,Candy Store,Thrift / Vintage Store,Café/CoffeeShop


### Which areas are the best for cafés or coffee shops?

In [17]:
#Find the top 5 ZIP codes with the highest frequency of Cafés/Coffee Shops
HI_grouped.sort_values(by=['Café/CoffeeShop'], axis=0, ascending=False).head()

,ZIP,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Baseball Field,Basketball Court,Beach,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Dog Run,Donut Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Indian Restaurant,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lingerie Store,Liquor Store,Locksmith,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Post Office,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Surf Spot,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volcano,Wine Bar,Women's Store,Yoga Studio,Zoo,Café/CoffeeShop
25,96859,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.166667
12,96813,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.166667,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.000000,0.166667
22,96850,0.0,0.0,0.047619,0.0,0.000000,0.0,0.0,0.047619,0.0,0.000000,0.000

In [18]:
#Obtain data on all 'Café' and 'Coffee Shop' venues in these 5 ZIP codes
ZIP = ['96859', '96822', '96850', '96860', '96815']
Coffee =['Café', 'Coffee Shop']
HIvenues_filtered = HI_venues[HI_venues['ZIP'].isin(ZIP)]
HIvenues_filtered = HIvenues_filtered[HIvenues_filtered['Venue Category'].isin(Coffee)]
HIvenues_filtered

,ZIP,ZIP Latitude,ZIP Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
161,96815,21.273462,-157.822444,Banyan Breeze,21.275116,-157.821405,Coffee Shop
162,96815,21.273462,-157.822444,Tucker & Bevvy Picnic Food,21.271676,-157.822416,Café
183,96815,21.273462,-157.822444,Starbucks,21.271720,-157.822633,Coffee Shop
187,96815,21.273462,-157.822444,Kai Coffee Hawaii,21.276026,-157.825260,Café
200,96815,21.273462,-157.822444,Knots Coffee Roasters,21.272010,-157.821170,Café
288,96850,21.303768,-157.862663,OEC Cafe,21.305648,-157.861344,Café
293,96850,21.303768,-157.862663,Starbucks,21.306136,-157.865161,Coffee Shop
298,96850,21.303768,-157.862663,kissaten,21.300785,-157.862791,Café
318,96859,21.360460,-157.890628,Starbucks,21.359091,-157.889553,Coffee Shop
337,96860,21.355655,-157.941232,green beans coffee kiosk,21.357018,-157.939691,Coffee Shop


In [19]:
#Visualize the top Café/Coffee shop venue clusters
map_coffee = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, name in zip(HIvenues_filtered['Venue Latitude'], HIvenues_filtered['Venue Longitude'], HIvenues_filtered['Venue']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_coffee)  
    
map_coffee

### Trending Venues

In [20]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '610055c3aa867d042aedacb6'},
 'response': {'venues': []}}

In [24]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

trending_venues_df

'No trending venues are available at the moment!'

In [23]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
venues_map

'Cannot generate visual as no trending venues are available at the moment!'